In [10]:
import pandas as pd
from collections import defaultdict

# Global tracking of seller activity
seller_stats = defaultdict(lambda: {'total': 0, 'fraud': 0})

In [1]:
!pip install pyspark

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("FraudStream") \
    .getOrCreate()

In [3]:
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, TimestampType

# Define schema to ensure correct dtypes
schema = StructType([
    StructField("transaction_id", StringType(), True),
    StructField("seller_id", StringType(), True),
    StructField("buyer_id", StringType(), True),
    StructField("amount", DoubleType(), True),
    StructField("payment_method", StringType(), True),
    StructField("device_type", StringType(), True),
    StructField("location", StringType(), True),
    StructField("timestamp", TimestampType(), True),
    StructField("is_fraud", IntegerType(), True)
])

df = spark.read.csv("data/transactions.csv", header=True, schema=schema)
df = df.orderBy("timestamp")
df.show(5)

+--------------+---------+--------+-------+--------------+-----------+---------+--------------------+--------+
|transaction_id|seller_id|buyer_id| amount|payment_method|device_type| location|           timestamp|is_fraud|
+--------------+---------+--------+-------+--------------+-----------+---------+--------------------+--------+
|       T089558|     S185|   B2277| 831.11|   Net Banking|     Tablet|   Mumbai|2025-03-07 21:04:...|       0|
|       T013074|     S136|   B1109|4354.84|   Net Banking|    Android|   Others|2025-03-07 21:05:...|       0|
|       T009809|     S122|   B8432|1262.13|    Debit Card|        iOS|  Chennai|2025-03-07 21:07:...|       0|
|       T042449|     S127|   B4396| 3165.2|           UPI|     Tablet|Bangalore|2025-03-07 21:08:...|       0|
|       T058474|     S172|   B5532|3405.24|           COD|    Desktop|Bangalore|2025-03-07 21:08:...|       0|
+--------------+---------+--------+-------+--------------+-----------+---------+--------------------+--------+
o

In [4]:
import pandas as pd
import xgboost as xgb
import pickle

# Load model & encoders
model = xgb.XGBClassifier()
model.load_model("model/xgb_fraud_model.json")
with open("model/label_encoders.pkl", "rb") as f:
    le_dict = pickle.load(f)

In [14]:
def process_batch(pdf):
    if pdf.empty:
        return

    def safe_transform(le, col_data):
        known_classes = set(le.classes_)
        return [le.transform([val])[0] if val in known_classes else -1 for val in col_data]

    # Handle unseen values safely
    for col in ['payment_method', 'device_type', 'location', 'seller_id']:
        if col in pdf:
            le = le_dict[col]
            pdf[col] = safe_transform(le, pdf[col])

    X_live = pdf[['seller_id', 'amount', 'payment_method', 'device_type', 'location']]
    preds = model.predict(X_live)
    pdf['predicted_fraud'] = preds

    # Update seller stats
    for _, row in pdf.iterrows():
        sid = row['seller_id']
        seller_stats[sid]['total'] += 1
        if row['predicted_fraud'] == 1:
            seller_stats[sid]['fraud'] += 1

    # Detect frauds
    frauds = pdf[pdf['predicted_fraud'] == 1]
    if not frauds.empty:
        print("⚠️ Fraudulent transactions detected:")
        print(frauds[['transaction_id', 'amount', 'seller_id']])

        # Save to log
        frauds.to_csv('fraud_logs/fraud_batch.csv', mode='a', header=False, index=False)

    # Print Top Risky Sellers
    print("\n📈 Top Risky Sellers (RWSI):")
    rwsi_df = pd.DataFrame([
        {'seller_id': sid, 'RWSI': round((v['fraud'] / v['total']) * 100, 2)}
        for sid, v in seller_stats.items() if v['total'] > 5
    ]).sort_values(by='RWSI', ascending=False).head(5)

    print(rwsi_df)

In [17]:
import os
os.makedirs('fraud_logs', exist_ok=True)

# Convert to Pandas and simulate stream
full_df = df.toPandas()
batch_size = 5000

for i in range(0, len(full_df), batch_size):
    batch = full_df.iloc[i:i+batch_size]
    print(f"Processing batch {i // batch_size + 1}...")
    process_batch(batch)

Processing batch 1...


<ipython-input-14-32f126b493cf>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pdf[col] = safe_transform(le, pdf[col])
<ipython-input-14-32f126b493cf>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pdf[col] = safe_transform(le, pdf[col])
<ipython-input-14-32f126b493cf>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

⚠️ Fraudulent transactions detected:
     transaction_id   amount  seller_id
9           T008308  1064.78          0
25          T074674  3203.99         19
53          T060990   206.13         40
63          T016976  4720.33          7
70          T002924  7028.71         26
...             ...      ...        ...
4864        T035346  2394.38         34
4868        T075580  2495.45         53
4905        T099335  3236.70         91
4918        T026826  3282.24         58
4967        T041409  3886.89         91

[324 rows x 3 columns]

📈 Top Risky Sellers (RWSI):
    seller_id   RWSI
37         57  27.84
44         47  27.41
72         48  27.21
22         54  24.95
60         35  21.18
Processing batch 2...


<ipython-input-14-32f126b493cf>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pdf[col] = safe_transform(le, pdf[col])
<ipython-input-14-32f126b493cf>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pdf[col] = safe_transform(le, pdf[col])
<ipython-input-14-32f126b493cf>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

⚠️ Fraudulent transactions detected:
     transaction_id   amount  seller_id
5010        T000197  1708.69         78
5034        T099847  2354.73         52
5065        T068115  3237.27         69
5070        T084129  1097.15         64
5079        T094696  3929.24         90
...             ...      ...        ...
9900        T067929  3855.89         91
9911        T033788  2434.60         57
9915        T070381  2891.46         54
9953        T001073  7481.24         92
9990        T050383  3281.00         47

[319 rows x 3 columns]

📈 Top Risky Sellers (RWSI):
    seller_id   RWSI
37         57  27.98
44         47  27.46
72         48  27.09
22         54  24.58
65         53  21.04
Processing batch 3...


...

⚠️ Fraudulent transactions detected:
      transaction_id   amount  seller_id
85007        T047221  4886.16         24
85020        T000418   682.46          0
85022        T015055  1348.17         61
85050        T092511  4824.60          6
85079        T000554  1833.30         49
...              ...      ...        ...
89909        T015588  3199.23         20
89924        T083612  2381.48         57
89929        T003834  3144.57         62
89940        T049482  3340.66         35
89986        T012585  1967.78          0

[338 rows x 3 columns]

📈 Top Risky Sellers (RWSI):
    seller_id   RWSI
37         57  28.46
72         48  27.92
44         47  27.57
22         54  24.47
60         35  20.97
Processing batch 19...


<ipython-input-14-32f126b493cf>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pdf[col] = safe_transform(le, pdf[col])
<ipython-input-14-32f126b493cf>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pdf[col] = safe_transform(le, pdf[col])
<ipython-input-14-32f126b493cf>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

⚠️ Fraudulent transactions detected:
      transaction_id   amount  seller_id
90025        T028783  2571.19         53
90026        T000403  6131.19         98
90028        T000884  7043.99         35
90052        T042224  4733.40          4
90054        T015372  4242.30         46
...              ...      ...        ...
94908        T036264  2478.75         50
94911        T036462  2846.47         53
94926        T042945  1625.63         95
94962        T003525  2974.33         54
94991        T000998  6284.78         10

[336 rows x 3 columns]

📈 Top Risky Sellers (RWSI):
    seller_id   RWSI
37         57  28.07
72         48  27.78
44         47  27.59
22         54  24.68
60         35  21.35
Processing batch 20...


<ipython-input-14-32f126b493cf>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pdf[col] = safe_transform(le, pdf[col])
<ipython-input-14-32f126b493cf>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pdf[col] = safe_transform(le, pdf[col])
<ipython-input-14-32f126b493cf>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

⚠️ Fraudulent transactions detected:
      transaction_id   amount  seller_id
95070        T086919  1686.01         62
95077        T059140  1383.71         48
95093        T019114  2876.21         57
95107        T061783  1826.83         74
95108        T079468  1724.60         50
...              ...      ...        ...
99967        T001564  2238.22          1
99968        T002889   411.80         47
99989        T002418  4873.68         26
99990        T000520  6575.33         12
99999        T060124  3130.45         19

[353 rows x 3 columns]

📈 Top Risky Sellers (RWSI):
    seller_id   RWSI
37         57  28.11
72         48  27.76
44         47  27.53
22         54  24.82
60         35  21.36
